## Практическое задание к уроку № 1 по теме "Введение, примеры задач, бизнес- и ML-метрики."

*1) Приведите еще примеры метрик для оценки рекомендаций/ранжирования (можно взять из интернета, или ваши знания)*

*2) Доделать все функции, где стоит комментарий "сделать дома"*

#### Задание 1.

### 1. Среднеквадратичная ошибка

Используется в случае, когда мы хотим предсказать, какой рейтинг  
пользователь поставит конкретному товару.

$MSE = mean((P-R)^2),$
где: 
- P - предсказанное значение;  
- R - истинное значение.

#### Задание 2.

## ML-mетрики качества

In [1]:
import numpy as np

### 1. Hit rate

In [2]:
def hit_rate(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return (flags.sum() > 0) * 1

In [3]:
def hit_rate_at_k(recommended_list, bought_list, k=5):
#   сделать в домашней работе
    return hit_rate(recommended_list[:k], bought_list)

In [4]:
recommended_list = [143, 156, 1134, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]

In [5]:
hit_rate(recommended_list, bought_list)

1

In [6]:
hit_rate_at_k(recommended_list, bought_list, k=3)

1

### 2. Precision

In [7]:
recommended_list = [2, 3, 8, 6, 9, 5, 10] #id товаров
bought_list = [1, 5, 6]

In [8]:
def precision(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)

In [9]:
precision(recommended_list, bought_list)

0.2857142857142857

In [10]:
def precision_at_k(recommended_list, bought_list, k=5):
    return precision(recommended_list[:k], bought_list)

In [11]:
precision_at_k(recommended_list, bought_list, k=5)

0.2

In [12]:
def money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5):
    
    recommended_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    
    flags = np.isin(recommended_list, bought_list)
    
    precision = np.dot(flags, prices_recommended).sum() / prices_recommended.sum()
    
    return precision

In [13]:
prices = [23, 2233, 534, 56, 1]
money_precision_at_k(recommended_list, bought_list, prices)

0.01966982788900597

### 3. Recall

In [14]:
def recall(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(bought_list)
    

def recall_at_k(recommended_list, bought_list, k=5):
#    сделать дома
    return recall(recommended_list[:k], bought_list)


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    # сделать дома
    recommended_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    
    flags = np.isin(recommended_list, bought_list)
    
    return np.dot(flags, prices_recommended).sum() / prices_bought.sum()

In [15]:
recall(recommended_list, bought_list)

0.6666666666666666

In [16]:
recall_at_k(recommended_list,bought_list,k=5)

0.3333333333333333

### 4. AP@k

In [17]:
recommended_list = [1, 21, 22, 4, 234, 232432, 234234, 666] #id товаров
bought_list = [1, 221, 3, 4,]

In [18]:
precision(recommended_list, bought_list)

0.25

In [19]:
recall(recommended_list, bought_list)

0.5

In [20]:
def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    if len(relevant_indexes) == 0:
        return 0
    
    amount_relevant = len(relevant_indexes)
            
    sum_ = sum([precision_at_k(recommended_list, bought_list, k=index_relevant+1) for index_relevant in relevant_indexes])
    return sum_/amount_relevant

In [21]:
ap_k(recommended_list, bought_list, k=5)

0.75

### 5. MAP@k

In [22]:
# теперь список из 3 пользователей
recommended_list_3_users = [[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43], 
                    [1134, 533, 14, 4, 15, 1543, 1, 99, 27, 3345],
                    [991, 3345, 27, 533, 43, 143, 1543, 156, 1134, 11]
                    ]

bought_list_3_users = [[521, 32, 143],  # юзер 1
                       [143, 156, 991, 43, 11], # юзер 2
                       [1,2]] # юзер 3

In [23]:
def map_k(recommended_list, bought_list, k=5):
    
    # сделать дома
    return np.array([ap_k(rec_list, buy_list, k=k) for rec_list, buy_list \
                     in zip(recommended_list, bought_list)]).mean()

In [24]:
map_k(recommended_list_3_users, bought_list_3_users, k=5)

0.3333333333333333

### 6. Normalized discounted cumulative gain ( NDCG@k)

In [25]:
# по желанию
def ndcg_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    bought_fact = np.isin(recommended_list, bought_list)
    
    i = np.arange(1, k+1)
    discount = np.where(i <= 2, i, np.log2(i))
    
    dcg_at_k = (bought_fact / discount).sum() / k
    ideal_dcg = (np.ones(discount.shape) / discount).sum() / k
    
    return dcg_at_k / ideal_dcg

In [26]:
ndcg_at_k(recommended_list, bought_list, k=5)

0.489938890671454

### 7. Mean Reciprocal Rank ( MRR@k )

In [27]:
def reciprocal_rank(recommended_list, bought_list, k=1):
    
    # сделать дома 
    mrrs = np.array([])
    
    for rec_list, buy_list in zip(recommended_list, bought_list):
        rec_list = np.array(rec_list)[:k]
        buy_list = np.array(buy_list)
        
        flags = np.isin(rec_list, buy_list)
        mrrs = np.append(mrrs, 1 / (np.where(flags)[0][0] + 1) \
                         if np.where(flags)[0].size > 0 else 0)
    
    return mrrs.mean()

In [28]:
reciprocal_rank(recommended_list_3_users, bought_list_3_users, k=5)

0.3333333333333333